In [1]:
import cryo
import polars as pl
import binascii
import web3
import json
from eth_abi import decode
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

In [2]:
# Functions
def bytes_to_hexstr(b: any) -> str:
    if isinstance(b,list):
        return [bytes_to_hexstr(a) for a in b]
    return '0x' + b.hex()

In [3]:
# Function Signatures 4 bytes
getBlocknumber_4b = '42cbb15c'

In [4]:
txs = cryo.collect('transactions', blocks=['latest'], rpc ='https://eth.merkle.io')

In [5]:
txs.head()

block_number,transaction_index,transaction_hash,nonce,from_address,to_address,value_binary,value_string,value_f64,input,gas_limit,gas_used,gas_price,transaction_type,max_priority_fee_per_gas,max_fee_per_gas,success,n_input_bytes,n_input_zero_bytes,n_input_nonzero_bytes,chain_id
u32,u64,binary,u64,binary,binary,binary,str,f64,binary,u64,u64,u64,u32,u64,u64,bool,u32,u32,u32,u64
22018587,0,"b""\xea`\x19\x982U\xe0\xd1\xc3x]\xe7\xdc\xb1\x85\xdb\x9c7\xd9\x1b\x9e\xb3d\x01L\x01rp\x0eA\xee\xbc""",886717,"b""\x93y;\xd1\xf3\xe3Z\x0e\xfd\x09\x8c0\xe4\x86\xa8`\xa0\xefuQ""","b""h\xd3\xa9s\xe7'.\xb3\x88\x02*\e\x18\xd9\xb2\xa2\xe6o\xbf""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01O\xfa\x1b""","""22018587""",2.2018587e7,"b""\xa0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Nh\xcc\xd3\xe8\x9fQ\xc3\x07L\xa5\x07+\xba\xc7s\x96\x0d\xfa6\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",305818,152762,861209480320,2,838277245009,861209480320,true,164,109,55,1
22018587,1,"b""\x13\xcd\x80g\x7f\xf4\xa3\xf1\x83\x1d\xcd)\xda\x9bI\x1bqk\x98}`\xf0-\x84\x8d\xaf.\xbe\x7fv\xb2J""",886718,"b""\x93y;\xd1\xf3\xe3Z\x0e\xfd\x09\x8c0\xe4\x86\xa8`\xa0\xefuQ""","b""h\xd3\xa9s\xe7'.\xb3\x88\x02*\e\x18\xd9\xb2\xa2\xe6o\xbf""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01O\xfa\x1b""","""22018587""",2.2018587e7,"b""\xa0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x88\xe6\xa0\xc2\xdd\xd2o\xee\xb6O\x03\x9a,A)o\xcb?V@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",367018,177808,953554052673,2,930621817362,953554052673,true,164,108,56,1
22018587,2,"b""\x97\x93P:\x83\xc9ZD+9\xbb\xd6{ic0y4W\xacX\x13\xff\xd8\x92\xfe!\x87\xb4\xb8\xd1\xd3""",886719,"b""\x93y;\xd1\xf3\xe3Z\x0e\xfd\x09\x8c0\xe4\x86\xa8`\xa0\xefuQ""","b""h\xd3\xa9s\xe7'.\xb3\x88\x02*\e\x18\xd9\xb2\xa2\xe6o\xbf""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01O\xfa\x1b""","""22018587""",2.2018587e7,"b""\x00\x00\x00i\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00R\xc7{\x0c\xb8'\xaf\xba\xd0""\xe6\xd6\xca\xf2\xc4DR\xed\xbc9\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",223330,131181,119873898486,2,96941663175,119873898486,true,164,109,55,1
22018587,3,"b""&\x8f\xed)C\x89\x89$\xdaS\x9dz\xc2\xe0:\x88\xa2\xb6\xe6\xaa\x93Y\xfb=\xbfVF\xc3M\xe0Zk""",886720,"b""\x93y;\xd1\xf3\xe3Z\x0e\xfd\x09\x8c0\xe4\x86\xa8`\xa0\xefuQ""","b""h\xd3\xa9s\xe7'.\xb3\x88\x02*\e\x18\xd9\xb2\xa2\xe6o\xbf""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01O\xfa\x1b""","""22018587""",2.2018587e7,"b""\x00\x00\x00i\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00%d~\x01\xbd\x09g\xc1\xb9Y\x9f\xa3R\x199\x87\x1d\x1d\x08\x88\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",160256,99692,77351902499,2,54419667188,77351902499,true,164,105,59,1
22018587,4,"b""a\xec?DmG\xfdk3\xbf}\x8a\xf9\xe5\xf8\x05\xc9sdF\xab\xfcb(]\xb5\xddN[\xc2\xf2\xd5""",886721,"b""\x93y;\xd1\xf3\xe3Z\x0e\xfd\x09\x8c0\xe4\x86\xa8`\xa0\xefuQ""","b""h\xd3\xa9s\xe7'.\xb3\x88\x02*\e\x18\xd9\xb2\xa2\xe6o\xbf""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01O\xfa\x1b""","""22018587""",2.2018587e7,"b""\xa0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xb8\x15\xef\xb8\xf5\x81\x19J\xe7\x90\x06\xd2N\x0d\x81Kv\x97\xf6\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",365858,176589,161464414207,2,138532178896,161464414207,true,164,109,55,1


## Balance

In [6]:
# ERC20s
USDT_ADDR ="0xdac17f958d2ee523a2206206994597c13d831ec7"
USDC_ADDR="0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48" 
# WALLET = Binance 18
RANDOM_WALLET="0xf70da97812CB96acDF810712Aa562db8dfA3dbEF" 
# block 
BLOCKS="21922375"


In [7]:
usdc_balance = cryo.collect('erc20_balances', blocks=[BLOCKS], contract=['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'], address=['0xf70da97812CB96acDF810712Aa562db8dfA3dbEF'], rpc='https://eth.merkle.io')

In [8]:
usdc_balance

block_number,erc20,address,balance_binary,balance_string,balance_f64,chain_id
u32,binary,binary,binary,str,f64,u64
21922375,"b""\xa0\xb8i\x91\xc6!\x8b6\xc1\xd1\x9dJ.\x9e\xb0\xce6\x06\xebH""","b""\xf7\x0d\xa9x\x12\xcb\x96\xac\xdf\x81\x07\x12\xaaV-\xb8\xdf\xa3\xdb\xef""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00n\x1a)\x8f\x80""","""472885333888""",4.7289e11,1


In [9]:
output_addr=usdc_balance['erc20'][0]

In [10]:
output_addr.hex()

'a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'

In [11]:
usdc_balance['address'][0].hex()

'f70da97812cb96acdf810712aa562db8dfa3dbef'

In [12]:
usdc_balance['balance_binary'][0]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00n\x1a)\x8f\x80'

In [13]:
balance= usdc_balance['balance_binary'][0]

In [14]:
int((balance).hex(),16)

472885333888

In [15]:
usdc_balance['balance_string'][0]

'472885333888'

In [16]:
balance_raw = decode(['uint256'], usdc_balance['balance_binary'][0])

In [17]:
balance_raw

(472885333888,)

In [18]:
 balance_raw[0] / 10**6

472885.333888

In [19]:
# BLOCKS="21922375" >> usdc 472,885.333888

## Aave Pools Data

In [107]:
#  AAVE
# https://etherscan.io/address/0x87870bca3f3fd6335c3f4ce8392d69350b4fa4e2#readProxyContract

In [60]:
MULTICALL3_ADDRESS = '0xcA11bde05977b3631167028862bE2a173976CA11'
MULTICALL3_ABI=json.loads('[{"inputs":[{"internalType":"bool","name":"requireSuccess","type":"bool"},{"components":[{"internalType":"address","name":"target","type":"address"},{"internalType":"bytes","name":"callData","type":"bytes"}],"internalType":"struct Multicall3.Call[]","name":"calls","type":"tuple[]"}],"name":"tryAggregate","outputs":[{"components":[{"internalType":"bool","name":"success","type":"bool"},{"internalType":"bytes","name":"returnData","type":"bytes"}],"internalType":"struct Multicall3.Result[]","name":"returnData","type":"tuple[]"}],"stateMutability":"payable","type":"function"}]')

In [61]:
AAVE_POOL_PROVIDER_ADDRESS = "0x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e"
AAVE_POOL_DATA_PROVIDER_ADDRESS = "0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3"
RATE_STRATEGY_V2= "0x9ec6F08190DeA04A54f8Afc53Db96134e5E3FdFB"
DAI_ADDRESS = "0x6b175474e89094c44da98b954eedeac495271d0f"  # DAI on Ethereum Mainnet
WETH_ADDRESS = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'  # WETH
USDC_ADDRESS = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'   # USDC

In [62]:
AAVE_POOL_DATA_PROVIDER_ABI = json.loads('[{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveData","outputs":[{"internalType":"uint256","name":"unbacked","type":"uint256"},{"internalType":"uint256","name":"accruedToTreasuryScaled","type":"uint256"},{"internalType":"uint256","name":"totalAToken","type":"uint256"},{"internalType":"uint256","name":"totalStableDebt","type":"uint256"},{"internalType":"uint256","name":"totalVariableDebt","type":"uint256"},{"internalType":"uint256","name":"liquidityRate","type":"uint256"},{"internalType":"uint256","name":"variableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"stableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"averageStableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"liquidityIndex","type":"uint256"},{"internalType":"uint256","name":"variableBorrowIndex","type":"uint256"},{"internalType":"uint40","name":"lastUpdateTimestamp","type":"uint40"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveTokensAddresses","outputs":[{"internalType":"address","name":"aTokenAddress","type":"address"},{"internalType":"address","name":"stableDebtTokenAddress","type":"address"},{"internalType":"address","name":"variableDebtTokenAddress","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getAllReservesTokens","outputs":[{"components":[{"internalType":"string","name":"symbol","type":"string"},{"internalType":"address","name":"tokenAddress","type":"address"}],"internalType":"struct IPoolDataProvider.TokenData[]","name":"","type":"tuple[]"}],"stateMutability":"view","type":"function"}]')


In [63]:
# Funct Sig
getAllReservesTokens_4b='b316ff89'
getReserveData_4b='35ea6a75'
getReserveTokensAddresses_4b='d2493b6c'

In [300]:
# getReserveData(address) method Response ]
#   unbacked   uint256 :  0
#   accruedToTreasuryScaled   uint256 :  23618787780361774928
#   totalAToken   uint256 :  2039801843604753838311618
#   totalStableDebt   uint256 :  0
#   totalVariableDebt   uint256 :  1764755511852628454833269
#   liquidityRate   uint256 :  19086348066645041248157072
#   variableBorrowRate   uint256 :  25954495065727705440586566
#   stableBorrowRate   uint256 :  0
#   averageStableBorrowRate   uint256 :  0
#   liquidityIndex   uint256 :  1040724217363078276332975414
#   variableBorrowIndex   uint256 :  1064692592486700566822429320
#   lastUpdateTimestamp   uint40 :  1740762059

# unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp

# output_types = ['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40']

In [64]:
# web3 instance, function from web3py
w3 = web3.Web3()
a3 = w3.eth.contract(address = AAVE_POOL_DATA_PROVIDER_ADDRESS, abi=AAVE_POOL_DATA_PROVIDER_ABI)
m3 = w3.eth.contract(address = MULTICALL3_ADDRESS, abi=MULTICALL3_ABI)

In [65]:
tokens = [
    '0x6B175474E89094C44Da98b954EedeAC495271d0F',  # DAI
    '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',  # WETH
    '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'   # USDC
]

In [66]:
calls = []
for token in tokens:
    # Encode the call to getReserveData for this token
    calldata = a3.encode_abi("getReserveData", args=[token])
    calls.append([AAVE_POOL_DATA_PROVIDER_ADDRESS, calldata])

In [67]:
calls

[['0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
  '0x35ea6a750000000000000000000000006b175474e89094c44da98b954eedeac495271d0f'],
 ['0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
  '0x35ea6a75000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'],
 ['0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
  '0x35ea6a75000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48']]

In [68]:
# result = m3.functions.tryAggregate(False, calls).call()
calldata_aave_pool = m3.encode_abi("tryAggregate", args=[False, calls])

In [69]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['latest'], 
}
            
eth_call_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata_aave_pool],
     output_format="polars",
    **cryo_kwargs,
)

In [70]:
calldata = eth_call_df['output_data'][0]

In [71]:
decoded_data = decode(['(bool,bytes)[]'] , calldata)[0]

In [72]:
decoded_data

((True,
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\xccD\xb5v\x16#\xa5C\xe4\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8cM\xe6[\xe9\x8c^T\xaav\xec\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00d\xcaK\xdf*\xf4\xc7\xc5\xdd\xc7g\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1d\x93o\xf2\xa4\xe00\x05[\xe2)\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x006\xe5\xd0\xc2\x96\xec^\xfdE#r\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [73]:
token_data = {
    'DAI_RESERVE_DATA': [],
    'WETH_RESERVE_DATA': [],
    'USDC_RESERVE_DATA': []
}

#### DAI Pools Data

In [74]:
dai_reserve_data = decoded_data[0]

In [75]:
data_dai=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],dai_reserve_data[1])

In [76]:
token_data['DAI_RESERVE_DATA'] = list(data_dai)

In [77]:
token_data

{'DAI_RESERVE_DATA': [0,
  8490453267109216863204,
  169617486339345064137488108,
  0,
  121847899577742672355247975,
  35755101714744415885189673,
  66367267128188838359671666,
  0,
  0,
  1119761759606039860342279125,
  1175297541637175565664921135,
  1741638935],
 'WETH_RESERVE_DATA': [],
 'USDC_RESERVE_DATA': []}

In [78]:
# [unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp
# ]=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],dai_reserve_data[1])

####  WETH Pools Data

In [79]:
weth_reserve_data = decoded_data[1]

In [80]:
data_weth=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],weth_reserve_data[1])

In [81]:
token_data['WETH_RESERVE_DATA'] = list(data_weth)

In [82]:
# [unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp
# ]=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],weth_reserve_data[1])

In [83]:
token_data

{'DAI_RESERVE_DATA': [0,
  8490453267109216863204,
  169617486339345064137488108,
  0,
  121847899577742672355247975,
  35755101714744415885189673,
  66367267128188838359671666,
  0,
  0,
  1119761759606039860342279125,
  1175297541637175565664921135,
  1741638935],
 'WETH_RESERVE_DATA': [0,
  29893041655513597218,
  2059494481538932427098712,
  0,
  1859304644024501182887407,
  37803274799172484814726411,
  49263713122627422769483288,
  0,
  0,
  1041289421605800102360240357,
  1065463288085244887131116316,
  1741639283],
 'USDC_RESERVE_DATA': []}

####  USDC Pools Data

In [84]:
usdc_reserve_data = decoded_data[2]

In [85]:
data_usdc=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],usdc_reserve_data[1])

In [86]:
token_data['USDC_RESERVE_DATA'] = list(data_usdc)

In [87]:
# [unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp
# ]=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],usdc_reserve_data[1])

In [88]:
token_data

{'DAI_RESERVE_DATA': [0,
  8490453267109216863204,
  169617486339345064137488108,
  0,
  121847899577742672355247975,
  35755101714744415885189673,
  66367267128188838359671666,
  0,
  0,
  1119761759606039860342279125,
  1175297541637175565664921135,
  1741638935],
 'WETH_RESERVE_DATA': [0,
  29893041655513597218,
  2059494481538932427098712,
  0,
  1859304644024501182887407,
  37803274799172484814726411,
  49263713122627422769483288,
  0,
  0,
  1041289421605800102360240357,
  1065463288085244887131116316,
  1741639283],
 'USDC_RESERVE_DATA': [0,
  38413372068,
  2949660285347877,
  0,
  1658798924361716,
  26296889048791639544969889,
  51957289161537487528548337,
  0,
  0,
  1123172715898789313973391483,
  1161531107182590183798749533,
  1741639283]}

### Aave Pool Data

In [89]:
field_names = [
    'unbacked', 'accruedToTreasuryScaled', 'totalAToken', 'totalStableDebt', 
    'totalVariableDebt', 'liquidityRate', 'variableBorrowRate', 'stableBorrowRate', 
    'averageStableBorrowRate', 'liquidityIndex', 'variableBorrowIndex', 'lastUpdateTimestamp'
]

In [90]:
df = pd.DataFrame(token_data, index=field_names)

In [91]:
df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,8490453267109216863204,29893041655513597218,38413372068
totalAToken,169617486339345064137488108,2059494481538932427098712,2949660285347877
totalStableDebt,0,0,0
totalVariableDebt,121847899577742672355247975,1859304644024501182887407,1658798924361716
liquidityRate,35755101714744415885189673,37803274799172484814726411,26296889048791639544969889
variableBorrowRate,66367267128188838359671666,49263713122627422769483288,51957289161537487528548337
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1119761759606039860342279125,1041289421605800102360240357,1123172715898789313973391483


In [92]:
readable_df = df

In [93]:
readable_df.loc['lastUpdateTimestamp'] = readable_df.loc['lastUpdateTimestamp'].apply(
    lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')
)

In [94]:
readable_df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,8490453267109216863204,29893041655513597218,38413372068
totalAToken,169617486339345064137488108,2059494481538932427098712,2949660285347877
totalStableDebt,0,0,0
totalVariableDebt,121847899577742672355247975,1859304644024501182887407,1658798924361716
liquidityRate,35755101714744415885189673,37803274799172484814726411,26296889048791639544969889
variableBorrowRate,66367267128188838359671666,49263713122627422769483288,51957289161537487528548337
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1119761759606039860342279125,1041289421605800102360240357,1123172715898789313973391483


In [95]:
readable_df.loc['totalAToken', 'DAI_RESERVE_DATA'] = readable_df.loc['totalAToken', 'DAI_RESERVE_DATA'] / 1e18
readable_df.loc['totalAToken', 'WETH_RESERVE_DATA'] = readable_df.loc['totalAToken', 'WETH_RESERVE_DATA'] / 1e18
readable_df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] / 1e18
readable_df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] / 1e18

In [96]:
readable_df.loc['totalAToken', 'USDC_RESERVE_DATA'] = readable_df.loc['totalAToken', 'USDC_RESERVE_DATA'] / 1e6
readable_df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] / 1e6

In [97]:
readable_df.loc['totalAToken'] = readable_df.loc['totalAToken'].apply(lambda x: f"{float(x):,.4f}" if isinstance(x, (int, float, str)) else x)
readable_df.loc['totalVariableDebt'] = readable_df.loc['totalVariableDebt'].apply(lambda x: f"{float(x):,.4f}" if isinstance(x, (int, float, str)) else x)


In [98]:
# Convert rates from ray (10^27) to percentage for all tokens
readable_df.loc['liquidityRate'] = readable_df.loc['liquidityRate'] / 1e27 * 100
readable_df.loc['variableBorrowRate'] = readable_df.loc['variableBorrowRate'] / 1e27 * 100

In [99]:
# Format the rate fields with 4 decimal places and add % sign
readable_df.loc['liquidityRate'] = readable_df.loc['liquidityRate'].apply(lambda x: f"{float(x):.4f}%" if isinstance(x, (int, float, str)) else x)
readable_df.loc['variableBorrowRate'] = readable_df.loc['variableBorrowRate'].apply(lambda x: f"{float(x):.4f}%" if isinstance(x, (int, float, str)) else x)

In [402]:
# readable_df.loc['totalAToken'] = df.loc['totalAToken'].apply(lambda x: f"{x:,.4f}")
# readable_df.loc['totalVariableDebt'] = df.loc['totalVariableDebt'].apply(lambda x: f"{x:,.4f}")
# readable_df.loc['liquidityRate'] = df.loc['liquidityRate'].apply(lambda x: f"{x:.4f}%")
# readable_df.loc['variableBorrowRate'] = df.loc['variableBorrowRate'].apply(lambda x: f"{x:.4f}%")

In [100]:
readable_df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,8490453267109216863204,29893041655513597218,38413372068
totalAToken,"169,617,486.3393","2,059,494.4815","2,949,660,285.3479"
totalStableDebt,0,0,0
totalVariableDebt,"121,847,899.5777","1,859,304.6440","1,658,798,924.3617"
liquidityRate,3.5755%,3.7803%,2.6297%
variableBorrowRate,6.6367%,4.9264%,5.1957%
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1119761759606039860342279125,1041289421605800102360240357,1123172715898789313973391483


In [59]:
df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,8264682939905964535187,28086699020048129266,37421718113
totalAToken,"169,620,759.4482","2,065,362.9894","2,942,226,109.6963"
totalStableDebt,0,0,0
totalVariableDebt,"121,397,888.8033","1,862,977.4122","1,663,523,858.5996"
liquidityRate,3.7578%,3.2949%,2.8144%
variableBorrowRate,7.0010%,4.2975%,5.5310%
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1119756753014401535780706695,1041283877456323856619372431,1123168856004783924142674086
